# V1 program

```javascript
var a
func f() {
    a = 5
    var b
    b = 6
}
func g() {
    var c
    c = 7
}
f()
g()
```

# V1 Grammar

```yaml
program: (statement | func_def)*
statement: definition | attribution | func_call
definition: "var" NAME
attribution: NAME "=" NUMBER
func_call: NAME()
func_def: "func" NAME() "{" statement* "}"
```

## T1 - Variable errors

```go
var a
var a
func f() {
    var a
    var b
}
func g() {
    var b
    c = 0
}
```
```
variable redefined: a
variable redefined: a
clearing local_symbol_table: ['b']
variable unknown: c
clearing local_symbol_table: ['b']
analysis ended

global_symbol_table: {'a': 0}
local_symbol_table: []
function_table: {'f()': 2, 'g()': 6}
```

## T2 - Function errors

```go
func f() {
}
func f() {
}
g()
```
```
function redefined: f()
function unknown: g()
analysis ended

global_symbol_table: {}
local_symbol_table: []
function_table: {'f()': 0}
```

## T3 - Sequential function call

```go
var a
func f() {
    a = 5
    var b
    b = 6
}
func g() {
    var c
    c = 7
}
f()
g()
```
```
clearing local_symbol_table: ['b']
clearing local_symbol_table: ['c']
analysis ended

global_symbol_table: {'a': 0}
local_symbol_table: []
function_table: {'f()': 1, 'g()': 6}
```
```
f() called in line 10
a at address 0 receives 5
b at address 1 receives 6
memory before removal of local variables: [5, 6]
deleting last activation frame: {'b': 1}
return to line 10
g() called in line 11
c at address 1 receives 7
memory before removal of local variables: [5, 7]
deleting last activation frame: {'c': 1}
return to line 11
execution ended

memory: [5]
call_stack: []
activation_frames: []
```

## T4 - Nested function call

```go
var a
func f() {
    a = 5
    var b
    b = 6
}
func g() {
    var c
    c = 7
    f()
}
g()
```
```
clearing local_symbol_table: ['b']
clearing local_symbol_table: ['c']
analysis ended

global_symbol_table: {'a': 0}
local_symbol_table: []
function_table: {'f()': 1, 'g()': 6}
```
```
g() called in line 11
c at address 1 receives 7
f() called in line 9
a at address 0 receives 5
b at address 2 receives 6
memory before removal of local variables: [5, 7, 6]
deleting last activation frame: {'b': 2}
return to line 9
memory before removal of local variables: [5, 7]
deleting last activation frame: {'c': 1}
return to line 11
execution ended

memory: [5]
call_stack: []
activation_frames: []
```

## Analyzer

In [79]:
program = """
var a
func f() {
    a = 5
    var b
    b = 6
}
func g() {
    var c
    c = 7
    f()
}
g()
"""

In [80]:
import rich

local_symbol_table = []

def analyze(p):
    memory_adress = 0
    is_local = False
    for line_number, line in enumerate(p.strip().split('\n')):
        match line.split():
            case ['var', name]:
                if is_local:
                    if name in local_symbol_table or name in global_symbol_table:
                        rich.print('[red1]variable redefined:', name)
                    else:
                        local_symbol_table.append(name)
                else:
                    if name in global_symbol_table:
                        rich.print('[red1]variable redefined:', name)
                    else:
                        global_symbol_table[name] = memory_adress
                        memory_adress += 1
            case [name, '=', number]:
                if is_local:
                    if name not in local_symbol_table and name not in global_symbol_table:
                        rich.print('[red1]variable unknown:', name)
                else:
                    if name not in global_symbol_table:
                        rich.print('[red1]variable unknown:', name)
            case ['func', name, '{']:
                is_local = True
                local_symbol_table = []
                if name in function_table:
                    rich.print('[red1]function redefined:', name)
                else:
                    function_table[name] = line_number
            case ['}']:
                if len(local_symbol_table) > 0:
                    print('clearing local_symbol_table:', local_symbol_table)
                is_local = False
            case [name] if name.endswith('()'):
                if name not in function_table:
                    rich.print('[red1]function unknown:', name)
            case _:
                rich.print('[red1]unmatched:', line)
    print('analysis ended\n')

global_symbol_table = {}
function_table = {}
analyze(program)
print('global_symbol_table:', global_symbol_table)
print('local_symbol_table:', local_symbol_table)
print('function_table:', function_table)

clearing local_symbol_table: ['b']
clearing local_symbol_table: ['c']
analysis ended

global_symbol_table: {'a': 0}
local_symbol_table: []
function_table: {'f()': 1, 'g()': 6}


In [87]:
def execute(p):
    pc = 0 # program counter
    call_stack = []
    activation_frames = []
    memory = [0] * len(global_symbol_table)
    is_local = False
    memory_address = len(memory)
    lines = p.strip().split('\n')
    while pc < len(lines):
        match lines[pc].split():
            case ['var', name]:
                if is_local and name not in global_symbol_table:
                    memory.append(0)
                    activation_frames[-1][name] = memory_address
                    print('created local', name, 'with address', memory_address)
                    memory_address += 1
            case [name, '=', number]:
                if is_local and name not in global_symbol_table:
                    address = activation_frames[-1][name]
                else:
                    address = global_symbol_table[name]
                memory[address] = int(number)
                print(name, 'at address', address, 'receives', number)
            case ['func', name, '{']:
                while lines[pc] != '}':
                    pc += 1
            case [name] if name.endswith('()'):
                activation_frames.append({})
                call_stack.append(pc)
                print(name, 'called in line', pc)
                pc = function_table[name]
                is_local = True
            case ['}']:
                is_local = False
                print('memory before removal of local variables:', memory[:memory_address])
                frame = activation_frames.pop()
                print('deleting last activation frame:', frame)
                memory_address = memory_address - len(frame)
                memory[:memory_address]
                pc = call_stack.pop()
                print('return to line', pc)
        pc += 1
    print('execution ended\n')
    
    print('memory:', memory[:memory_address])
    print('call_stack:', call_stack)
    print('activation frames:', activation_frames)
    
execute(program)


g() called in line 11
created local c with address 1
c at address 1 receives 7
f() called in line 9
a at address 0 receives 5
created local b with address 2
b at address 2 receives 6
memory before removal of local variables: [5, 7, 6]
deleting last activation frame: {'b': 2}
return to line 9
memory before removal of local variables: [5, 7]
deleting last activation frame: {'c': 1}
return to line 11
execution ended

memory: [5]
call_stack: []
activation frames: []
